In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

In [2]:
import torch
from torch import nn
import torch.utils.data

In [3]:
import numpy as np

## 実装
- クロスエントロピー
- 仮説

In [4]:
criterion = nn.CrossEntropyLoss() # 損失の定義

In [5]:
preds = torch.tensor([[0.2, 0.8]]) # 二値分類
labels = torch.tensor([1]) # 正解ラベル

loss = criterion(preds, labels)
print(loss)

tensor(0.4375)


## 演習
- 3×3のモデル
- クロスエントロピー

In [6]:
X = torch.tensor(np.arange(12).reshape(4, 3)).float()
Y = torch.tensor([1, 2, 0, 1])
model = nn.Linear(3, 3)

In [7]:
criterion(model(X), Y)

tensor(1.2623, grad_fn=<NllLossBackward>)

## Pytorchで
- 勾配降下
- ニュートン法

で ルート2を求める

- 勾配降下の場合
  - $f(x) = x^3 - 6x$で計算する

In [8]:
def F(x):
    return x ** 3 -  6 * x

In [21]:
x = torch.tensor([0.01], requires_grad=True)
y = F(x)

In [22]:
learning_rate = 0.1

In [23]:
for i in range(10):
    y = F(x)
    z = torch.autograd.grad(y, x)
    x = x - learning_rate * z[0]
    print(i, "loop", x)

0 loop tensor([0.6100], grad_fn=<SubBackward0>)
1 loop tensor([1.0984], grad_fn=<SubBackward0>)
2 loop tensor([1.3364], grad_fn=<SubBackward0>)
3 loop tensor([1.4006], grad_fn=<SubBackward0>)
4 loop tensor([1.4121], grad_fn=<SubBackward0>)
5 loop tensor([1.4139], grad_fn=<SubBackward0>)
6 loop tensor([1.4142], grad_fn=<SubBackward0>)
7 loop tensor([1.4142], grad_fn=<SubBackward0>)
8 loop tensor([1.4142], grad_fn=<SubBackward0>)
9 loop tensor([1.4142], grad_fn=<SubBackward0>)


- ニュートン法の場合
  - $f(x) = x^2 - 2$で計算する

In [13]:
def G(x):
    return x ** 2 - 2

In [25]:
x = torch.tensor([0.1], requires_grad=True)
y = G(x)

In [26]:
y

tensor([-1.9900], grad_fn=<SubBackward0>)

In [27]:
for i in range(10):
    y = G(x)
    y.backward()
    # backwardで値を入れる時はdataに代入
    # xに代入すると,backwardの対象でなくなる
    x.data = x.data - y/ x.grad
    print(i, "lootp", x)
    x.grad.zero_()

0 lootp tensor([10.0500], requires_grad=True)
1 lootp tensor([5.1245], requires_grad=True)
2 lootp tensor([2.7574], requires_grad=True)
3 lootp tensor([1.7414], requires_grad=True)
4 lootp tensor([1.4449], requires_grad=True)
5 lootp tensor([1.4145], requires_grad=True)
6 lootp tensor([1.4142], requires_grad=True)
7 lootp tensor([1.4142], requires_grad=True)
8 lootp tensor([1.4142], requires_grad=True)
9 lootp tensor([1.4142], requires_grad=True)


## Softmax回帰の実装
- irisに対し,softmax回帰で実装する

In [7]:
def train_epoch(model, data_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader): # 入力と正解
         optimizer.zero_grad() # Weightの初期化
         output = model(data) # 仮説で値代入
         loss = criterion(output, target) # 損失
         loss.backward() # 微分の計算
         optimizer.step() # パラメータの更新
         if batch_idx % 10 == 0:
             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                 batch_idx, batch_idx * len(data), len(data_loader.dataset),
                 100. * batch_idx / len(data_loader), loss.item()))

def valid_epoch(model, data_loader):
    model.train()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader): # 入力と正解
             optimizer.zero_grad() # Weightの初期化
             output = model(data) # 仮説で値代入
             output.dtype
             loss = criterion(output, target) # 損失
             # 本来は全体でロスを数えて荷重平均を取る,accuracyを計算する

             if batch_idx % 10 == 0:
                 print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                     batch_idx, batch_idx * len(data), len(data_loader.dataset),
                     100. * batch_idx / len(data_loader), loss.item()))

In [8]:
iris = load_iris()
X_train, X_valid, y_train, y_valid = train_test_split(iris.data, iris.target, test_size=0.2)

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train)
X_valid = torch.tensor(X_valid).float()
y_valid = torch.tensor(y_valid)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
valid_dataset = torch.utils.data.TensorDataset(X_valid, y_valid)

model = nn.Linear(4, 3)

batch_size  = 120 # ミニバッチのデータの数
max_epoch = 100 #
train_loader = torch.utils.data.DataLoader(train_dataset, 
                   batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(train_dataset, 
                   batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss() # 損失の定義
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #(確率的)勾配降下法

In [9]:
for epoch in range(max_epoch):
    train_epoch(model, train_loader)
    valid_epoch(model, valid_loader)

Train Epoch: 0 [0/120 (0%)]	Loss: 1.360266
Test Epoch: 0 [0/120 (0%)]	Loss: 1.268214
Train Epoch: 0 [0/120 (0%)]	Loss: 1.268214
Test Epoch: 0 [0/120 (0%)]	Loss: 1.194083
Train Epoch: 0 [0/120 (0%)]	Loss: 1.194083
Test Epoch: 0 [0/120 (0%)]	Loss: 1.134569
Train Epoch: 0 [0/120 (0%)]	Loss: 1.134568
Test Epoch: 0 [0/120 (0%)]	Loss: 1.087408
Train Epoch: 0 [0/120 (0%)]	Loss: 1.087408
Test Epoch: 0 [0/120 (0%)]	Loss: 1.050642
Train Epoch: 0 [0/120 (0%)]	Loss: 1.050642
Test Epoch: 0 [0/120 (0%)]	Loss: 1.022376
Train Epoch: 0 [0/120 (0%)]	Loss: 1.022377
Test Epoch: 0 [0/120 (0%)]	Loss: 1.000799
Train Epoch: 0 [0/120 (0%)]	Loss: 1.000799
Test Epoch: 0 [0/120 (0%)]	Loss: 0.984276
Train Epoch: 0 [0/120 (0%)]	Loss: 0.984276
Test Epoch: 0 [0/120 (0%)]	Loss: 0.971439
Train Epoch: 0 [0/120 (0%)]	Loss: 0.971439
Test Epoch: 0 [0/120 (0%)]	Loss: 0.961214
Train Epoch: 0 [0/120 (0%)]	Loss: 0.961214
Test Epoch: 0 [0/120 (0%)]	Loss: 0.952797
Train Epoch: 0 [0/120 (0%)]	Loss: 0.952797
Test Epoch: 0 [0/120 (

## AdaptivePooling
毎回挙動が怪しくなるので

In [94]:
m = nn.AdaptiveMaxPool2d(3)

In [89]:
x = torch.tensor(np.arange(24).reshape(1, 3, 8).astype(float))
output = m(x)

In [90]:
input.shape

torch.Size([1, 3, 8])

In [91]:
output.shape

torch.Size([1, 3, 3])

In [92]:
x

tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11., 12., 13., 14., 15.],
         [16., 17., 18., 19., 20., 21., 22., 23.]]], dtype=torch.float64)

In [93]:
output

tensor([[[ 1.0000,  3.5000,  6.0000],
         [ 9.0000, 11.5000, 14.0000],
         [17.0000, 19.5000, 22.0000]]], dtype=torch.float64)